# Installation

In [ ]:
! pip install transformers Pillow torch scipy matplotlib torchvision datasets diffusers accelerate vector_quantize_pytorch pytube moviepy torchmetrics timm

# Classes

In [ ]:
from PIL import Image
import torch
import torchvision.transforms as transforms
import torchaudio
from torch.utils.data import Dataset
import json

class ImMuTe(Dataset):

    def __init__(self, images_path, captions_json_file, audios_path, start=0, end=2999, sampling_rate=16000, pixel=64, normalize=False):
        super().__init__()
        self.images_path = images_path
        self.captions_json_file = captions_json_file
        self.audios_path = audios_path
        self.sampling_rate = sampling_rate

        # define transform to convert image to tensors
        transform = [
            transforms.Resize(pixel),
            transforms.ToTensor(),
        ]
        if normalize:
          transform.append(transforms.Normalize([0.5], [0.5]))

        self.transform = transforms.Compose(transform)

        # load captions
        with open(self.captions_json_file, "r", encoding='utf-8') as f:
            caps_dict = json.load(f)

        # preload all data in a dict
        self.all_data = {}
        index = 0
        for i in range(len(caps_dict)):
            if i < start:
              continue
            if i > end:
              break
            try:
              # load audio
              wav, sr = torchaudio.load(f"{self.audios_path}/aud_{i}.wav")
              wav = torchaudio.functional.resample(wav, orig_freq=sr, new_freq=self.sampling_rate)
              wav = torch.mean(wav, dim=0, keepdim=True)
              if wav.size(-1) < self.sampling_rate * 10:
                  pad_len = self.sampling_rate * 10 - wav.size(-1)
                  wav = torchfunc.pad(wav, (0, pad_len))

              # transform image
              img = Image.open(f"{self.images_path}/test_{i}.png")
              img = self.transform(img)

              # index image-text pair and save them to dict
              self.all_data[index] = (wav, img, caps_dict[str(i)])
              index += 1
            except:
                continue


    def __len__(self):

        # get total length of dataset
        length = len(self.all_data)

        return length

    def __getitem__(self, idx):

        # get image-text pair by index
        wav, img, txt = self.all_data[idx]

        return (wav, img, txt)


# Initialisation

In [ ]:
import torch
from transformers import AutoProcessor, AutoImageProcessor, AutoModel

musicgen_proc = AutoProcessor.from_pretrained("facebook/musicgen-small", low_cpu_mem_usage=True)
vit_proc = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k", low_cpu_mem_usage=True)

i2m = AutoModel.from_pretrained("juliagsy/imagic", trust_remote_code=True)
i2m.model.load_state_dict(torch.load("imagic_state_dict.bin"))

# Training

In [ ]:
from torch.utils.data import DataLoader

immute = ImMuTe("images", "caption.json", "audios", start=0, end=5000, sampling_rate=32000, pixel=256)

In [ ]:
immute_ds = DataLoader(immute, shuffle=True, batch_size=32)

In [ ]:
from torch.optim import Adam
from IPython.display import Audio
from IPython.core.display import display

i2m.to("cuda")
print()

In [ ]:
opt = Adam(list(i2m.model.musicgen.decoder.lm_heads.parameters()) + list(i2m.model.img_lin.parameters()), lr=1e-05, weight_decay=1e-02)

for epoch in range(0, 32):
    losses = []
    for b in immute_1_ds:
        wav, img, _ = b
        wav = wav.squeeze(1).tolist()

        img = vit_proc(img, do_rescale=False, return_tensors="pt")
        wav = musicgen_proc(
            audio=wav,
            sampling_rate=32000,
            padding=True,
            return_tensors="pt",
        )

        wav = wav.to("cuda")
        img = img.to("cuda")


        opt.zero_grad()
        loss = i2m.forward(img, wav)

        losses.append(loss.item())
        loss.backward()

        torch.nn.utils.clip_grad_norm_(list(i2m.model.musicgen.decoder.lm_heads.parameters()) + list(i2m.model.img_lin.parameters()), 0.5)
        opt.step()

    print(f"epoch {epoch}: {np.mean(losses)}")
